In [1]:
import spacy
import nltk
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from textdistance import jaccard, sorensen_dice, cosine, levenshtein, lcsseq, ratcliff_obershelp
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import textdistance
import numpy as np


for i in range(0,2):
    
    # Load the Spacy model and create a TfidfVectorizer
    # nlp = spacy.load('en_core_web_lg') if i == 0 else spacy.load("en_core_web_md")
    # print('en_core_web_lg' if i == 0 else 'en_core_web_md', '\n')
    nlp = spacy.load('en_core_web_lg' if i == 0 else 'en_core_web_md')
    print(nlp.meta['name'], '\n')


    # Define two sample sentences
    # sentence1 = "The quick brown fox jumps over the lazy dog"
    # sentence2 = "A quick brown dog jumps over the lazy cat"
    sentence1 = "Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that involves the development of algorithms and models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. In other words, ML involves training a machine to identify patterns in data and use those patterns to make predictions or decisions."
    sentence2 = "Machine Learning can also refer to the process of training a machine learning model on a dataset to make predictions or decisions. This involves selecting an appropriate algorithm, preparing the data, and tuning the parameters of the model to achieve the desired level of accuracy. The ultimate goal of ML is to create models that can generalize well to new, unseen data and make accurate predictions or decisions in real-world scenarios."
    # sentence1 = "Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that involves the development of algorithms and models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. In other words, ML involves training a machine to identify patterns in data and use those patterns to make predictions or decisions."
    # sentence2 = "Environmental Studies(EVS full form) is the study of the environment, which refers to the surroundings in which all live. Everything that directly or indirectly promotes human survival is considered part of the environment. Plants, animals, and soil, water, and air are examples of living and non-living components. The living components interact with their surroundings and adjust to the changing conditions."
    # sentence1 = "computer science is a type of artificial intelligence (AI) that allows software applications to become more accurate at predicting outcomes without being explicitly programmed to do so. Machine learning algorithms use historical data as input to predict new output values."
    # sentence2 = "computer science is the study of computers and computational systems. Unlike electrical and computer engineers, computer scientists deal mostly with software and software systems; this includes their theory, design, development, and application."
    # sentence1 = "The bright and radiant sun shone down upon the lush green fields filling them with warmth and life"
    # sentence2 = "The brilliant and glowing star cast its brilliant light onto the verdant meadows imbuing them with vitality and heat"
    # sentence1 = "Social media has revolutionized the way we communicate and interact with each other."
    # sentence2 = "The rise of social media has transformed the landscape of communication and interpersonal relationships."
    # sentence1 =  "Artificial intelligence (AI) is a field of computer science that focuses on creating machines that can perform tasks that typically require human intelligence."
    # sentence2 = "AI involves developing algorithms and models that enable machines to perform intelligent tasks, such as natural language processing and computer vision."
    # sentence1 = "The internet has become an essential tool for communication, research, and commerce in the modern era."
    # sentence2 = "In today's society, the internet plays a critical role in connecting people, providing information, and facilitating economic transactions."
    # sentence1 = "The Earth's atmosphere consists of several layers that protect the planet from harmful radiation and provide the air we breathe."
    # sentence2 = "The Earth's atmosphere is composed of several layers that help shield the planet from harmful cosmic rays and provide the necessary gases for life to thrive."
    # sentence1 = "Mobile devices have transformed the way we consume media, with many people now using smartphones and tablets to watch videos, listen to music, and browse the web."
    # sentence2 = "The widespread adoption of mobile devices has led to a shift in how we access and consume media, with many people now preferring to watch videos, listen to music, and browse the internet on their smartphones and tablets."

    """Nearly similar but below one is more accurate"""
    # Preprocess the sentences
    # def preprocess_sentence(sentence):
    #     # sentence = sentence.lower()
    #     sentence_tokens = nlp(sentence.lower())
    #     sentence_tokens = nltk.word_tokenize(sentence.lower())
    #     # sentence_tokens = [token.lemma_ for token in sentence_tokens if not token.is_stop and not token.is_punct]
    #     sentence_tokens = [token for token in sentence_tokens if token not in stop_words]
    #     lemmatizer = WordNetLemmatizer()
    #     sentence_tokens = [lemmatizer.lemmatize(token) for token in sentence_tokens]
    #     sentence = " ".join(sentence_tokens)
    #     return sentence
    """End"""

    """More accurate one"""
    def preprocess_sentence(text):
        
        text = text.lower()
        text = text.translate(str.maketrans("", "", string.punctuation))
        stop_words = set(stopwords.words("english"))
        words = text.split()
        words = [w for w in words if w not in stop_words]
        return " ".join(words)



    processed_sentence1 = preprocess_sentence(sentence1)
    processed_sentence2 = preprocess_sentence(sentence2)


    # Compute Spacy similarity score
    doc1 = nlp(processed_sentence1)
    doc2 = nlp(processed_sentence2)
    spacy_similarity_score = doc1.similarity(doc2)


    # Compute TF-IDF vector representation for each sentence
    vectorizer = TfidfVectorizer()
    sentence1_tfidf = vectorizer.fit_transform([processed_sentence1])
    sentence2_tfidf = vectorizer.transform([processed_sentence2])


    # Compute cosine similarity score using TF-IDF vectors
    cosine_similarity_score = cosine_similarity(sentence1_tfidf, sentence2_tfidf)[0][0]


    # test
    ''' test for snippet 1 '''
    """use with try and expect"""
    
    try:
        
        tokens1 = word_tokenize(processed_sentence1)
        tokens2 = word_tokenize(processed_sentence2)
            
        # create word embeddings for each string
        model = gensim.models.Word2Vec([tokens1, tokens2], min_count=2)
        word_vectors = {word: model.wv[word] for word in model.wv.index_to_key}
            
        # calculate the average vector for each string
        vector1 = np.mean([word_vectors[word] for word in tokens1 if word in word_vectors], axis=0)
        vector2 = np.mean([word_vectors[word] for word in tokens2 if word in word_vectors], axis=0)
            
        # calculate cosine similarity between the vectors
        sim_score = cosine_similarity([vector1], [vector2])[0][0]
        
    except:
        
        sim_score = weighted_score

    """end"""
    # test


    # Compute Jaccard similarity score using set intersection
    # words1 = set(processed_sentence1.split())
    # words2 = set(processed_sentence2.split())
    # jaccard_similarity_score = len(words1.intersection(words2)) / len(words1.union(words2))

    tokens1 = set(nltk.word_tokenize(sentence1.lower()))
    tokens2 = set(nltk.word_tokenize(sentence2.lower()))

    jaccard_similarity_score = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))


    # Weighted average of the similarity scores
    weighted_score = (0.5 * spacy_similarity_score) + (0.3 * jaccard_similarity_score) + (0.2 * cosine_similarity_score)


    # test
    """Useless genrates use less values"""
    """begin"""
    # jaccard_score1 = jaccard(processed_sentence1, processed_sentence2)
    # sorensen_dice_score1 = sorensen_dice(processed_sentence1, processed_sentence2)
    # cosine_score1 = cosine(processed_sentence1, processed_sentence2)
    # levenshtein_score1 = levenshtein.normalized_distance(processed_sentence1, processed_sentence2)
    # ratcliff_obershelp_score1 = ratcliff_obershelp(processed_sentence1, processed_sentence2)
    # edit_distance = textdistance.levenshtein.normalized_distance(sentence1, sentence2)

    # print("Jaccard Index:", jaccard_score1)
    # print("Sørensen-Dice coefficient:", sorensen_dice_score1)
    # print("Cosine similarity:", cosine_score1)
    # print("Edit distance:", edit_distance)
    # print("Ratcliff/Obershelp similarity:", ratcliff_obershelp_score1)
    # print("Levenshtein distance:", levenshtein_score1,"\n\n\n")
    """end"""
    # test


    # Compute the average similarity score
    average_similarity_score = (spacy_similarity_score + cosine_similarity_score + jaccard_similarity_score + weighted_score + sim_score) / 5
    avg = spacy_similarity_score * .1 + jaccard_similarity_score*.2 + cosine_similarity_score*.2 + weighted_score*.2 + sim_score * .2
    ###
    avg1 = spacy_similarity_score * .15 + ( jaccard_similarity_score + cosine_similarity_score + ( weighted_score + sim_score ) * .15 ) / 2
    # test

    # test

    # Print the similarity scores
    print("Spacy similarity score:", spacy_similarity_score)
    print("Cosine similarity score:", cosine_similarity_score)
    print("Jaccard similarity score:",jaccard_similarity_score)
    print("Weighted similarity score:", weighted_score)
    print("Gensim similarity score:", sim_score)
    print("average ",average_similarity_score)
    print("\n\nAnswer score is :",avg1,"\n\n")


c:\Users\Hp\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


core_web_lg 

Spacy similarity score: 0.9514847092758087
Cosine similarity score: 0.7483314773547882
Jaccard similarity score: 0.2916666666666667
Weighted similarity score: 0.7129086501088621
Gensim similarity score: 0.8614414
average  0.7131665754462398


Answer score is : 0.7807980301971439 


core_web_md 

Spacy similarity score: 0.9546089382680001
Cosine similarity score: 0.7483314773547882
Jaccard similarity score: 0.2916666666666667
Weighted similarity score: 0.7144707646049577
Gensim similarity score: 0.8614414
average  0.7141038441438973


Answer score is : 0.7813838231331798 




In [5]:
def feron(question):
    print(question)
    return ["a1","a2","a3","a4"]

def getAnswers(questions,number):
    
    answers = []
    answers = feron(questions[number])
    return answers

getAnswers(["q1","q2","q3","q4"],1)
    

q2


['a1', 'a2', 'a3', 'a4']

In [25]:
import spacy
import nltk
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import numpy as np
from math import ceil

In [26]:
def getScore(sentence1,sentence2):
    
    def preprocess_sentence(text):
            
            text = text.lower()
            text = text.translate(str.maketrans("", "", string.punctuation))
            stop_words = set(stopwords.words("english"))
            words = text.split()
            words = [w for w in words if w not in stop_words]
            return " ".join(words)


    processed_sentence1 = preprocess_sentence(sentence1)
    processed_sentence2 = preprocess_sentence(sentence2)
    final_score = 0
    
    for i in range(1,3):
        
        nlp = spacy.load('en_core_web_lg' if i == 1 else 'en_core_web_md')

        # print(nlp.meta['name'], '\n')
        weighted_score = 0

        # Compute Spacy similarity score
        doc1 = nlp(processed_sentence1)
        doc2 = nlp(processed_sentence2)
        spacy_similarity_score = 0
        spacy_similarity_score = doc1.similarity(doc2)


        # Compute TF-IDF vector representation for each sentence
        vectorizer = TfidfVectorizer()
        sentence1_tfidf = vectorizer.fit_transform([processed_sentence1])
        sentence2_tfidf = vectorizer.transform([processed_sentence2])


        # Compute cosine similarity score using TF-IDF vectors
        cosine_similarity_score = 0
        cosine_similarity_score = cosine_similarity(sentence1_tfidf, sentence2_tfidf)[0][0]

        
        try:
            
            tokens1 = word_tokenize(processed_sentence1)
            tokens2 = word_tokenize(processed_sentence2)
                
            # create word embeddings for each string
            model = gensim.models.Word2Vec([tokens1, tokens2], min_count=2)
            word_vectors = {word: model.wv[word] for word in model.wv.index_to_key}
                
            # calculate the average vector for each string
            vector1 = np.mean([word_vectors[word] for word in tokens1 if word in word_vectors], axis=0)
            vector2 = np.mean([word_vectors[word] for word in tokens2 if word in word_vectors], axis=0)
                
            # calculate cosine similarity between the vectors
            sim_score = 0
            sim_score = cosine_similarity([vector1], [vector2])[0][0]
            # print("ends within try")
            
        except:
            
            sim_score = 0
            # print("except")
            sim_score = weighted_score


        tokens1 = set(nltk.word_tokenize(sentence1.lower()))
        tokens2 = set(nltk.word_tokenize(sentence2.lower()))

        jaccard_similarity_score = 0
        jaccard_similarity_score = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))


        # Weighted average of the similarity scores
        weighted_score = (0.5 * spacy_similarity_score) + (0.3 * jaccard_similarity_score) + (0.2 * cosine_similarity_score)


        # Compute the average similarity score
        average_similarity_score = 0
        average_similarity_score = spacy_similarity_score * .15 + ( jaccard_similarity_score + cosine_similarity_score + ( weighted_score + sim_score ) * .15 ) / 2
        final_score += average_similarity_score
        # # Print the similarity scores
        # print("Spacy similarity score:", spacy_similarity_score)
        # print("Cosine similarity score:", cosine_similarity_score)
        # print("Jaccard similarity score:",jaccard_similarity_score)
        # print("Weighted similarity score:", weighted_score)
        # print("Gensim similarity score:", sim_score)
        # # print("average ",average_similarity_score)
        # print("\n\nAnswer score is :",average_similarity_score,"\n\n")
    res = ceil((final_score/2)*100)
    # print("Simalarity score is :",0 if ceil((final_score/2)*100) < 50 else (100 if ceil((final_score/2)*100) > 100 else ceil((final_score/2)*100)))
    return 0 if res < 50 else (100 if res > 100 else res) 
    # return average_similarity_score


In [27]:
 # Define two sample sentences
# sentence1 = "The quick brown fox jumps over the lazy dog"
# sentence2 = "A quick brown dog jumps over the lazy cat"
# sentence1 = "Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that involves the development of algorithms and models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. In other words, ML involves training a machine to identify patterns in data and use those patterns to make predictions or decisions."
# sentence2 = "Machine Learning can also refer to the process of training a machine learning model on a dataset to make predictions or decisions. This involves selecting an appropriate algorithm, preparing the data, and tuning the parameters of the model to achieve the desired level of accuracy. The ultimate goal of ML is to create models that can generalize well to new, unseen data and make accurate predictions or decisions in real-world scenarios."
# sentence1 = "Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that involves the development of algorithms and models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. In other words, ML involves training a machine to identify patterns in data and use those patterns to make predictions or decisions."
# sentence2 = "Environmental Studies(EVS full form) is the study of the environment, which refers to the surroundings in which all live. Everything that directly or indirectly promotes human survival is considered part of the environment. Plants, animals, and soil, water, and air are examples of living and non-living components. The living components interact with their surroundings and adjust to the changing conditions."
# sentence1 = "computer science is a type of artificial intelligence (AI) that allows software applications to become more accurate at predicting outcomes without being explicitly programmed to do so. Machine learning algorithms use historical data as input to predict new output values."
# sentence2 = "computer science is the study of computers and computational systems. Unlike electrical and computer engineers, computer scientists deal mostly with software and software systems; this includes their theory, design, development, and application."
# sentence1 = "The bright and radiant sun shone down upon the lush green fields filling them with warmth and life"
# sentence2 = "The brilliant and glowing star cast its brilliant light onto the verdant meadows imbuing them with vitality and heat"
# sentence1 = "Social media has revolutionized the way we communicate and interact with each other."
# sentence2 = "The rise of social media has transformed the landscape of communication and interpersonal relationships."
# sentence1 =  "Artificial intelligence (AI) is a field of computer science that focuses on creating machines that can perform tasks that typically require human intelligence."
# sentence2 = "AI involves developing algorithms and models that enable machines to perform intelligent tasks, such as natural language processing and computer vision."
# sentence1 = "The internet has become an essential tool for communication, research, and commerce in the modern era."
# sentence2 = "In today's society, the internet plays a critical role in connecting people, providing information, and facilitating economic transactions."
sentence1 = "The Earth's atmosphere consists of several layers that protect the planet from harmful radiation and provide the air we breathe."
sentence2 = "The Earth's atmosphere is composed of several layers that help shield the planet from harmful cosmic rays and provide the necessary gases for life to thrive."
#sentence1 = "Mobile devices have transformed the way we consume media, with many people now using smartphones and tablets to watch videos, listen to music, and browse the web."
# sentence2 = "The widespread adoption of mobile devices has led to a shift in how we access and consume media, with many people now preferring to watch videos, listen to music, and browse the internet on their smartphones and tablets."
res = getScore(sentence1,sentence2)
print("Answer score :",res)

Answer score : 87
